In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import os
os.chdir('/m/nbe/project/rtmeg/problearn/mneflow/')
import mneflow



# Dataset parameters
dpath = '/m/nbe/work/zubarei1/collabs/bcicomp_ecog/sub1/'
fname = 'sub1_comp.mat'

import_opt = dict(fs=1000,
                  savepath='./tfr/',
                  out_name='bcig_ecog1_seq',
                  input_type='iid',
                  overwrite=True,
                  val_size=0.1,
                  array_keys={'X': 'train_data', 'y': 'train_dg'},
                  picks=None,
                  target_picks=None,
                  target_type='float',
                  segment=500,
                  augment=True,
                  aug_stride=50,
                  transpose=('X', 'y'),
                  scale=True,
                  decimate=None,
                  bp_filter=False,
                  transform_targets=True,
                  seq_length=10
                  )

meta = mneflow.produce_tfrecords([dpath+fname], **import_opt)

TypeError: produce_tfrecords() got an unexpected keyword argument 'val_size'

In [2]:
# batch the dataset according to that value
dataset = mneflow.Dataset(meta, train_batch=1000, class_subset=None, pick_channels=None, decim=None)

# training parameters
optimizer_params = dict(l1_lambda=3e-6,learn_rate=3e-4, task='regression')

optimizer = mneflow.Optimizer(**optimizer_params)
# model parameters
graph_specs = dict(n_ls=32,  # number of latent factors
                   filter_length=16,  # convolutional filter length
                   pooling=4,  # convlayer pooling factor
                   stride=4,  # stride parameter for pooling layer
                   padding='SAME',
                   nonlin=tf.nn.relu,
                   pool_type='max',
                   model_path=meta['savepath'],
                   dropout=.25,)

model = mneflow.models.VARCNN(dataset,optimizer,graph_specs)
#model = mneflow.models.LFCNNR(dataset,optimizer,graph_specs)

model.build()

results = model.train(early_stopping=3, min_delta=5e-6, n_iter=50000,
                         eval_step=50)

X0: (?, 62, 500)
de-mix init : OK
dmx (?, 500, 1, 32)
conv init
conv inint:OK shape: (?, 125, 1, 32)
fc ::: 4000 1
fc init : OK
X: (?, 62, 500, 1)
y_pred: (?, 1)
(1,)
L1 penalty applied to weights
Initialization complete!
i 0, tr_loss 2.51894, tr_acc -125.963 v_loss 3.31708, v_acc -160.549
i 50, tr_loss 0.887892, tr_acc -15.3941 v_loss 0.850267, v_acc -40.273
i 100, tr_loss 0.468449, tr_acc -2.96821 v_loss 0.55453, v_acc -25.8542


KeyboardInterrupt: 